# An example to use NBAgent as a deep searcher.

In [ ]:
# Since agent need to use some tools, we need to load necessary environment variables.
import dotenv

dotenv.load_dotenv()

In [ ]:
import os

from prompt import DEEP_SEARCHER_SYS_PROMPT
from nbagent.agents import NBAgent
from nbagent.models import OpenAIServerModel

In [ ]:
MODEL_ID = "enter your model_id here"
INFERENCE_ENDPOINT = "enter your inference endpoint here"
API_KEY = "enter your api_key here"

In [ ]:
task = "enter your task here"

In [ ]:
# Initialize the OpenAI server model
model = OpenAIServerModel(
    model_id=MODEL_ID,
    base_url=INFERENCE_ENDPOINT,
    api_key=API_KEY
)
# Initialize the NBAgent with the model and other configurations
deep_searcher = NBAgent(
    model=model,
    max_tokens=20000,
    max_response_length=3096,
    custom_system_prompt=DEEP_SEARCHER_SYS_PROMPT,
    tool_specs={
        "web_search_tool_factory": {"serper_api_key": os.getenv("SERPER_API_KEY"), "max_text_limitation": 1000},
        "jina_reader_tool_factory": {"api_key": os.getenv("JINA_API_KEY"), "token_budget": 100000},
    }
)

In [ ]:
step_iterator = deep_searcher.run(
    task=task,
    max_steps=20,
    stream=True,
)

In [ ]:
step_idx = 0
while True:
    step_idx += 1
    try:
        step = next(step_iterator)
        print("\n" + "="*60)
        print(f"🚀 Step {step_idx}")
        print("="*60)
        
        if not step.action.code or not step.action.think:
            print("\n📄 Full Response:")
            print("-" * 40)
            print(step.action.full_response)
            
        if step.action.think:
            print("\n💭 Thought:")
            print("-" * 40)
            print(step.action.think)
            
        if step.action.code:
            print("\n💻 Code:")
            print("-" * 40)
            print(step.action.code)
            
        if step.observations:
            print("\n👀 Observation:")
            print("-" * 40)
            print(step.observations)
            
    except StopIteration:
        print("\n" + "="*60)
        print("✅ Task completed successfully!")
        print("="*60)
        break